In [21]:
import torchvision
import torch

import torch.jit
import skimage
import numpy as np

import matplotlib.pyplot as plt


In [33]:


# Download ImageNet labels
!curl -o imagenet_classes.txt https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10472  100 10472    0     0  35023      0 --:--:-- --:--:-- --:--:-- 35140


In [28]:
def preprocess_image(image, channels=3):
    image_data = np.asarray(image).astype(np.float32)
    image_data = image_data.transpose([2, 0, 1]) # transpose to CHW
    mean = np.array([0.079, 0.05, 0]) + 0.406
    std = np.array([0.005, 0, 0.001]) + 0.224
    for channel in range(image_data.shape[0]):
        image_data[channel, :, :] = (image_data[channel, :, :] / 255 - mean[channel]) / std[channel]
    image_data = np.expand_dims(image_data, 0)
    return image_data


In [73]:
weights = torchvision.models.MobileNet_V3_Large_Weights.IMAGENET1K_V1
model = torchvision.models.mobilenet_v3_large(weights=weights)
model.eval()
v1 = torch.zeros(1, 3, 224, 224, requires_grad=True)
dynamic_axes = {'input_1': [0, 2, 3]}


Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /home/nikita/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth


  0%|          | 0.00/21.1M [00:00<?, ?B/s]

In [74]:
torch.onnx.export(model, v1, f='mobilenetv3_large.onnx', export_params=True,   do_constant_folding=True,
                  opset_version=12, input_names=["input_1"], output_names=["output_1"], dynamic_axes=dynamic_axes)


/home/nikita/venv/lib/python3.10/site-packages/torch/onnx/utils.py:2020: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input_1
  warnings.warn(


In [94]:
im1 = preprocess_image(skimage.io.imread('tests/f0.png'))
im2 = skimage.io.imread('tests/m2.png')

In [95]:

res=model(torch.Tensor(im1)).detach().numpy()[0]

In [72]:
plt.imshow(im1.transpose([0,1,2]))

ValueError: axes don't match array

In [96]:
[categories[x] for x in np.argsort(res)[-20:]]

['lakeside',
 'paddle',
 'red-breasted merganser',
 'soccer ball',
 'torch',
 'speedboat',
 'pool table',
 'tobacco shop',
 'Dandie Dinmont',
 'Bedlington terrier',
 'container ship',
 'pole',
 'ski',
 'tennis ball',
 'golf ball',
 'croquet ball',
 'bow',
 'racket',
 'ballplayer',
 'scoreboard']

'binder'